# Analysis of Enron Data
## Based on Udacity intro to machine learning course

### Data sources:
- Raw email text data can be found at: https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tgz
and a breakdown of emails by sender can be found [here](data/emails_by_address/)
- The financial data was compiled from [this file](data/financial_data.pdf)
- The persons of interest "pois" come from [this file](data/poi_names.txt)

### Get the data already preprocessed in [outlier_removal](outlier_removal.ipynb) and [imputing_data](imputing_data.ipynb)

In [1]:
import pickle

with open('imputer_dicts.pkl', 'rb') as f:
    imputer_dicts = pickle.load(f)

my_dataset = imputer_dicts['mean']

# Start with all features except: 'email_address'
features_list = ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments',\
'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi',\
'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances',\
'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income',\
'long_term_incentive', 'from_poi_to_this_person']

### Make a pipeline

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

pipe = Pipeline([
                 ('scaler', StandardScaler()),
                 ('polynomials', PolynomialFeatures(2)),
                 ('feature_selr', SelectFromModel(
                                                 ExtraTreesClassifier(
                                                     random_state=2,
                                                     class_weight='balanced'), 
                                                 threshold='mean')),
                 ('estimator', GaussianNB())
                ])

Get the pipeline parameters found in [parameter_optimization](parameter_optimization.ipynb)

In [13]:
with open('testing_params.pickle', 'rb') as f:
    testing_params = pickle.load(f)

with open('large_grid.pkl', 'rb') as f:
    large_grid = pickle.load(f)
    
# clf = pipe.set_params(**testing_params)
clf = large_grid.best_estimator_

In [14]:
clf

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('polynomials', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('feature_selr', SelectFromModel(estimator=ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
           criterion='gini'...random_state=42, splitter='best'),
          learning_rate=0.1, n_estimators=250, random_state=42))])

In [15]:
### Dump classifier, dataset, and features_list so anyone can check your results.
from tools.tester import dump_classifier_and_data
dump_classifier_and_data(clf, my_dataset, features_list)

In [16]:
run tools/tester.py

Loading data
Done loading
Start testing
Testing splits: 
. 5 % . 10 % . 20 % . 40 % . 80 % Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('polynomials', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('feature_selr', SelectFromModel(estimator=ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
           criterion='gini'...random_state=42, splitter='best'),
          learning_rate=0.1, n_estimators=250, random_state=42))])
	Accuracy: 0.81947	Precision: 0.27132	Recall: 0.21000	F1: 0.23675	F2: 0.21994
	Total predictions: 15000	True positives:  420	False positives: 1128	False negatives: 1580	True negatives: 11872

Done testing
